# Seq2Seq

이름 : 송예원

기수 : 11기

작성자 : 10 기 신재우

In [ ]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [ ]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 57.0 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [ ]:
path_to_folder = "/content/drive/MyDrive/DSL/과제/RNN"

In [ ]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv(path_train, index_col = 0)
val_dataset = pd.read_csv(path_val, index_col = 0)
test_dataset = pd.read_csv(path_test, index_col = 0)

In [ ]:
train_dataset

원문  \
0      'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1                                           씨티은행에서 일하세요?   
2                  푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   
3       11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.   
4         6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.   
...                                                  ...   
29995                             그는 충격받았는지 아무 표정이 없습니다.   
29996                      그는 충분히 비난받았고, 이제 내가 비난받을 차례야.   
29997                        그는 충청북도 청주시 상당구 서문동에 살고 있다.   
29998                                   그는 취미가 아무것도 없어요?   
29999                            그는 치료를 위해 병원으로 후송되었습니다.   

                                                     번역문  \
0      Bible Coloring' is a coloring application that...   
1                            Do you work at a City bank?   
2      PURITO's bestseller, which recorded 4th rough ...   
3      In Chapter 11 Jesus called Lazarus from the to...   
4      I would feel grateful to know how many stocks ...   
...                                                  ...   
29995  He looks shocked and doesn't have any facial e...   
29996   He was thoroughly criticized and now is my turn.   
29997  He is living in Seomun-dong, Sangdang-gu, Cheo...   
29998                       Doesn't he have any hobbies?   
29999      He was carried to the hospital for treatment.   

                                               en_tokens  \
0      ['<sos>', 'bible', 'coloring', "'", 'is', 'a',...   
1      ['<sos>', 'do', 'you', 'work', 'at', 'a', 'cit...   
2      ['<sos>', 'purito', "'s", 'bestseller', ',', '...   
3      ['<sos>', 'in', 'chapter', '11', 'jesus', 'cal...   
4      ['<sos>', 'i', 'would', 'feel', 'grateful', 't...   
...                                                  ...   
29995  ['<sos>', 'he', 'looks', 'shocked', 'and', 'do...   
29996  ['<sos>', 'he', 'was', 'thoroughly', 'criticiz...   
29997  ['<sos>', 'he', 'is', 'living', 'in', 'seomun'...   
29998  ['<sos>', 'does', "n't", 'he', 'have', 'any', ...   
29999  ['<sos>', 'he', 'was', 'carried', 'to', 'the',...   

                                               kr_tokens  
0      ['<sos>', "'", 'bible', 'coloring', "'", '은', ...  
1      ['<sos>', '씨티', '은행', '에서', '일하', '세요', '?', '...  
2      ['<sos>', '푸', '리', '토의', '베스트셀러', '는', '해외', ...  
3      ['<sos>', '11', '장', '에서', '는', '예수', '님', '이'...  
4      ['<sos>', '6.5', ',', '7', ',', '8', '사이즈', '가...  
...                                                  ...  
29995  ['<sos>', '그', '는', '충격', '받', '았', '는지', '아무'...  
29996  ['<sos>', '그', '는', '충분히', '비난', '받', '았', '고'...  
29997  ['<sos>', '그', '는', '충청북도', '청주시', '상당구', '서문동...  
29998  ['<sos>', '그', '는', '취미', '가', '아무것', '도', '없'...  
29999  ['<sos>', '그', '는', '치료', '를', '위하', '어', '병원'...  

[30000 rows x 4 columns]

# Vocabulary

In [ ]:
#딕셔너리 형태로 바꿔주기
#{원문:, 번역문:, en_tokens:, kr_tokens}
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [ ]:
train_dict[:1]

[{'원문': "'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.",
  '번역문': "Bible Coloring' is a coloring application that allows you to experience beautiful stories in the Bible.",
  'en_tokens': ['<sos>',
   'bible',
   'coloring',
   "'",
   'is',
   'a',
   'coloring',
   'application',
   'that',
   'allows',
   'you',
   'to',
   'experience',
   'beautiful',
   'stories',
   'in',
   'the',
   'bible',
   '.',
   '<eos>'],
  'kr_tokens': ['<sos>',
   "'",
   'bible',
   'coloring',
   "'",
   '은',
   '성경',
   '의',
   '아름',
   '답',
   'ㄴ',
   '이야기',
   '를',
   '체험',
   '하',
   'ㄹ',
   '수',
   '있',
   '는',
   '컬러링',
   '앱',
   '이',
   'ㅂ니다',
   '.',
   '<eos>']}]

In [ ]:
min_freq = 2
unk_token = "<unk>" #vocabulary에 없는 글자 처리
pad_token = "<pad>" #문장 길이 맞추기
sos_token = "<sos>" #start of sequence
eos_token = "<eos>" #end of sequence

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [ ]:
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [ ]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10465


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [ ]:
train_dataset['en_tokens']

0        [<sos>, bible, coloring, ', is, a, coloring, a...
1        [<sos>, do, you, work, at, a, city, bank, ?, <...
2        [<sos>, purito, 's, bestseller, ,, which, reco...
3        [<sos>, in, chapter, 11, jesus, called, lazaru...
4        [<sos>, i, would, feel, grateful, to, know, ho...
                               ...                        
29995    [<sos>, he, looks, shocked, and, does, n't, ha...
29996    [<sos>, he, was, thoroughly, criticized, and, ...
29997    [<sos>, he, is, living, in, seomun, -, dong, ,...
29998    [<sos>, does, n't, he, have, any, hobbies, ?, ...
29999    [<sos>, he, was, carried, to, the, hospital, f...
Name: en_tokens, Length: 30000, dtype: object

In [ ]:
en_indices = []
for word in train_dataset['en_tokens']:
  index = en_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  en_indices.append(tensor)

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  index = kr_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  kr_indices.append(tensor)

train_dataset['kr_indices'] = kr_indices

In [ ]:
en_indices = []
for word in val_dataset['en_tokens']:
  index = en_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  en_indices.append(tensor)

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  index = kr_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  kr_indices.append(tensor)

val_dataset['kr_indices'] = kr_indices

In [ ]:
en_indices = []
for word in test_dataset['en_tokens']:
  index = en_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  en_indices.append(tensor)

test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  index = kr_vocab.lookup_indices(word)
  tensor = torch.LongTensor(index)
  kr_indices.append(tensor)

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [ ]:
batch_no = 80

def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)
    en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [ ]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 25])
Shape of kr_indices: torch.Size([80, 35])
Sample en_indices batch: tensor([[   2, 6294,    8,   83,  390,   14,  356,   12,    8,  491,  608,  253,
           42, 2515,    4,    3,    0,    0,    0,    0,    0,    0,    0,    0,
            0]])
Sample kr_indices batch: tensor([[   2, 2751,    6,  420,    9,  138, 1951,   21,  582,   49,  489,   20,
         1066,   53,    4,    3,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])
Batch 2
Shape of en_indices: torch.Size([80, 27])
Shape of kr_indices: torch.Size([80, 37])
Sample en_indices batch: tensor([[   2,   29,    8,  707,    5,  291,  284,    6,   32,    9,    7, 2318,
            9,   27,    7, 1926,  102,    0,    5,  151,   11,    5, 1432,    4,
            3,    0,    0]])
Sample kr_indices batch: tensor([[   2, 1904,    7,   11, 3439, 1580,  969,   17, 2007,  159, 1519,    0,
          127,  414,   20,

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [ ]:
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super(Encoder, self).__init__()
    self.input_size = input_size #one-hot vectors size
    self.embedding_encoder_size = embedding_encoder_size
    self.hidden_size = hidden_size

    #Embedding Layer: Convert one-hot vectors into dense embedding size vectors
    self.embedding = nn.Embedding(num_embeddings = input_size, embedding_dim = embedding_encoder_size)
    #GRU
    self.gru = nn.GRU(input_size = embedding_encoder_size, hidden_size = hidden_size, batch_first = True)

  def forward(self, input):
    embedded = self.embedding(input) #Source sentence 넣어 embedding vector 만들기
    output, hidden = self.gru(embedded) #context vector 반환

    return output, hidden

In [ ]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding_decoder_size = embedding_decoder_size
    self.output_size = output_size

    self.embedding = nn.Embedding(output_size, embedding_decoder_size)
    self.gru = nn.GRU(embedding_decoder_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input)
    preds, hidden = self.gru(embedded,hidden)
    preds = self.fc(preds)

    return preds, hidden

In [ ]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device 가 들어갑니다.

  Init 단계에서는 정말로 할 것이 크게 없으며, forward 단계에서 for loop 과 if 조건문들을 활용해서 Teacher Forcing 을 적용시켜주면 됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Target : [Batch_size, Target Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]

  힌트 :
  - outputs 란 0 으로 채워진 텐서를 하나 만들며 디코더의 t-번째 아웃풀을 outputs 의 t-번째 칸에 넣으면서 for loop 을 진행해주시면 됩니다.
  - Target 의 첫번째 요소 (:, 0) 는 항상 <sos> 입니다! 이것을 디코더의 첫 Input 으로 넣어준 다음에 for loop 을 진행해주시면 됩니다.
  - random.random() < teacher_forcing_ratio 를 통해서 조건문을 만들 수가 있습니다!
  - Teacher Force 가 적용이 안될 때는 디코더의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 됩니다.
    이것들 중에서 가장 최댓값만 중요하기에 argmax 를 이용해서 최댓값의 인덱스를 다음 Decoder 의 Input 으로 넣어주면 됩니다.
  - squeeze 와 unsqueeze 를 적극적으로 활용해서 차원을 조정해주시면 됩니다!
  """
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):

    #최종 결과를 담을 output 텐서 객체 만들기:
    target_len = target.shape[1] #단어 개수
    batch_size = target.shape[0] #배치 크기
    vocab_size = self.decoder.output_size #출력 차원
    outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)

    #인코더를 거쳐 context vector 출력
    output, hidden = self.encoder(input)

    #target의 첫번째 요소가 <sos>가 decoder의 첫 input.
    input = target[:,0].unsqueeze(1)

    for t in range(1, target_len):
      preds, hidden = self.decoder(input, hidden)
      outputs[:,t,:] = preds.squeeze(1) #예측값 저장
      teacher_force = random.random() < teacher_forcing_ratio #teacher forcing 할지 말자 결정 (T or F)
      if teacher_force:
        input = target[:,t].unsqueeze(1)
      else:
        input = preds.argmax(2) #가장 확률 높은 단어의 index
        #input = input.unsqueeze(2)

    return outputs

# Training

In [ ]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,537,441 trainable parameters


### 하이퍼 파라미터 설정!

In [ ]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [ ]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.798491757074992, Val Loss: 2.886949486202664
Epoch: 2, Train Loss : 2.4706740881601967, Val Loss: 2.8982100032624745
Epoch: 3, Train Loss : 2.306828525543213, Val Loss: 2.8225818172333734
Epoch: 4, Train Loss : 2.1585843197504677, Val Loss: 2.8376546927860806
Epoch: 5, Train Loss : 1.9962840773264567, Val Loss: 2.749026650474185
Epoch: 6, Train Loss : 1.8358771966298422, Val Loss: 2.8560760021209717
Epoch: 7, Train Loss : 1.6898608258565266, Val Loss: 2.865084421067011
Epoch: 8, Train Loss : 1.5363476444880169, Val Loss: 2.912611367210509
Epoch: 9, Train Loss : 1.4132601210276285, Val Loss: 2.9999033458649165
Epoch: 10, Train Loss : 1.296182688554128, Val Loss: 3.144860661219037


파라미터 저장

In [ ]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [ ]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'soon', 'after', ',', 'it', 'was', 'occupied', 'until', 'daejeon', ',', 'and', 'the', 'government', 'established', 'the', '<unk>', 'river', 'defense', 'line', 'based', 'in', 'busan', 'and', 'daegu', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그리고', '<unk>', '대전', '까지', '도', '점령', '되', '어', '정부', '는', '부산', ',', '대구', '지역', '을', '<unk>', '으로', '하여', '<unk>', '<unk>', '을', '구축', '하', '게', '되', '어요', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '후', ',', ',', '바', '부', '부', '하', '을', '어', '<unk>', '<unk>', '에', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [ ]:
class Encoder(nn.Module):
  """
  이전과 Encoder 부분은 달라지는게 없습니다!
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super(Encoder, self).__init__()
    self.input_size = input_size #one-hot vectors size
    self.embedding_encoder_size = embedding_encoder_size
    self.hidden_size = hidden_size

    #Embedding Layer: Convert one-hot vectors into dense embedding size vectors
    self.embedding = nn.Embedding(num_embeddings = input_size, embedding_dim = embedding_encoder_size)
    #GRU
    self.gru = nn.GRU(input_size = embedding_encoder_size, hidden_size = hidden_size, batch_first = True)

  def forward(self, input):
    embedded = self.embedding(input) #Source sentence 넣어 embedding vector 만들기
    output, hidden = self.gru(embedded) #context vector 반환

    return output, hidden

In [ ]:
class Attention(nn.Module):
  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """

  def __init__(self, hidden_size):
    super(Attention,self).__init__()
    self.hidden_size = hidden_size
    self.softmax = nn.Softmax(dim=1)

  def forward(self, decoder_outputs, encoder_outputs):
    #print("Decoder Outputs Shape:", decoder_outputs.shape)
    #print("Encoder Outputs Shape:", encoder_outputs.shape)

    attention_scores = torch.bmm(decoder_outputs, encoder_outputs.transpose(1,2)) #scalar value
    attention_weights = self.softmax(attention_scores)

    #print("Attention_weights shape:", attention_weights.shape)

    attention_value = torch.sum(attention_weights.transpose(1,2) * encoder_outputs, dim=1)
    #attention_value = torch.sum(attention_weights * encoder_outputs.transpose(1,2), dim=1,keepdim=True)
    #print("Attention_value shape:", attention_value)
    return attention_value

In [ ]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding_decoder_size = embedding_decoder_size
    self.output_size = output_size

    self.attention = Attention(hidden_size)

    self.embedding = nn.Embedding(output_size, embedding_decoder_size)
    self.gru = nn.GRU(embedding_decoder_size, hidden_size, batch_first=True)
    self.W_c = nn.Linear(hidden_size*2,hidden_size)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input)
    decoder_outputs, hidden = self.gru(embedded,hidden) #GRU 지나고 난 후 output

    #print("Hidden shape:", hidden.shape)

    attention_value = self.attention(decoder_outputs, encoder_outputs) #attention_value
    #encoder_outputs = encoder_outputs.transpose(1,2)

    attention_value = attention_value.unsqueeze(1)

    #print("Attention Value Shape:", attention_value.shape)
    #print("Decoder Outputs Shape:", decoder_outputs.shape)

    v_t = torch.cat((attention_value.transpose(1,2),decoder_outputs.transpose(1,2)),dim=1) #concat
    #print("v_t shape",v_t.shape)

    output = torch.tanh(self.W_c(v_t.view(v_t.size(0),-1))) #matrix multiplication + activation
    output = self.fc(output) #linear
    output = output.unsqueeze(1)
    #print("output shape:", output.shape)

    return output, hidden

In [ ]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder,device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):
    #최종 결과를 담을 output 텐서 객체 만들기:
    target_len = target.shape[1] #단어 개수
    batch_size = target.shape[0] #배치 크기
    vocab_size = self.decoder.output_size #출력 차원
    outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)

    #인코더를 거쳐 context vector 출력
    encoder_outputs, hidden = self.encoder(input)

    #target의 첫번째 요소가 <sos>가 decoder의 첫 input.
    input = target[:,0].unsqueeze(1)

    for t in range(1, target_len):
      output, hidden = self.decoder(input, hidden, encoder_outputs)
      outputs[:,t,:] = output.squeeze(1) #예측값 저장
      teacher_force = random.random() < teacher_forcing_ratio #teacher forcing 할지 말자 결정 (T or F)
      if teacher_force:
        input = target[:,t].unsqueeze(1)
      else:
        input = output.argmax(2) #가장 확률 높은 단어의 index
        #input = input.unsqueeze(2)

    return outputs


# Training

In [ ]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [ ]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 32,635,617 trainable parameters


### 하이퍼 파라미터 설정!

In [ ]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [ ]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 3.6185882008870442, Val Loss: 3.6146503705827016
Epoch: 2, Train Loss : 3.6825295836130776, Val Loss: 3.6471991879599437
Epoch: 3, Train Loss : 3.6800279076894125, Val Loss: 3.5886323641216946
Epoch: 4, Train Loss : 3.699443994522095, Val Loss: 3.6722909268878756
Epoch: 5, Train Loss : 3.6852556953430176, Val Loss: 3.613274411549644
Epoch: 6, Train Loss : 3.6844203605651855, Val Loss: 3.611912984696646
Epoch: 7, Train Loss : 3.6918570353190105, Val Loss: 3.583168866142394
Epoch: 8, Train Loss : 3.6719884357452393, Val Loss: 3.6167490520174543
Epoch: 9, Train Loss : 3.682047123591105, Val Loss: 3.580993069542779
Epoch: 10, Train Loss : 3.6881834449768065, Val Loss: 3.6443208474961537


### 파라미터 저장

In [ ]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [ ]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'she', 'ca', "n't", 'move', 'the', 'desk', 'on', 'her', 'own', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그녀', '는', '직접', '책상', '을', '옮기', 'ㄹ', '수', '없', '습니다', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!